# Modelo de clasificación de imágenes: piedra, papel, tijeras.
Trabajo de la asignatura de "Deep Learning" del Máster en Data Science y Big Data de la Universidad de Sevilla.

## Sobre las referencias del trabajo
Las referencias de todos los archivos del trabajo están especificadas en el pdf adjunto.

Por ejemplo, se podrá encontrar 

        [C] https://stackoverflow.com/questions/32419510/how-to-get-reproducible-results-in-keras

que sigue la misma numeración que la indicada en las referencias del pdf (C, en este caso) y, además, se indica la url para facilitar el acceso.

## Importación de librerías <a class="anchor" id="librerias"></a>

In [ ]:
import numpy as np
import tensorflow as tf
import os 
import cv2
import random

# Importamos archivos .py auxiliares
import redes_neuronales as m # modelos con los que vamos a trabajar 
import funciones_estudio_modelo as f # funciones últiles para la visualización e interpretación de resultados

# Aumento de datos: clase Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Carga de datos
from sklearn.model_selection import train_test_split # partición train, validation, test

# Visualización del modelo
from tensorflow.keras.utils import plot_model

# Medidas de rendimiento y visualizaciones
from livelossplot import PlotLossesKerasTF # Plot de la función loss en cada época
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sns
import sklearn.metrics as skm
import pandas as pd

In [ ]:
tf.__version__

In [ ]:
# Semilla aleatoria
# [C] https://stackoverflow.com/questions/32419510/how-to-get-reproducible-results-in-keras

seed=33
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

## Carga de datos mediante la clase Sequence <a class="anchor" id="datos-sequence"></a>

### Clase Sequence <a class="anchor" id="clase-sequence"></a>

In [ ]:
# Referencias para la estructura de la clase Sequence:
# * Práctica 8 de la asignatura
# * [1] https://stackoverflow.com/questions/70230687/how-keras-utils-sequence-works
# * ChatGPT: ejemplo básico de funcionamiento añadiendo aumento de datos


class RockPaperScissorsSequence(tf.keras.utils.Sequence):
  # --- Atributos obligatorios de la clase ---
  # Inicialización
  def __init__(self, image_paths, labels, batch_size, image_size, shuffle=True, augment=False):
      self.image_paths = np.array(image_paths) # ruta de las imágenes
      self.labels = labels # etiquetas de las imágenes ('rock', 'paper' o 'scissors')

      # Corrección de la IA de Google Colab:
      # Como usamos model.compile(..., loss='sparse_categorical_crossentropy), las etiquetas deben ser numéricas. Vamos a mapearlo:
      self.class_mapping = {'rock': 0, 'paper': 1, 'scissors': 2} # Map string labels to numerical values

      self.batch_size = batch_size # número de imágenes en cada batch
      self.image_size = image_size # tamaño de imagen que se le da a la red
      self.data_length = len(self.image_paths) # número de imágenes

      self.shuffle = shuffle # mezclar imágenes (al final de cada época)
      # Preparamos índices y mezclamos si es necesario
      self.indexes = np.arange(len(self.image_paths))
      if self.shuffle:
          np.random.shuffle(self.indexes)

      self.augment = augment # modificar imágenes (reducción sobreajuste)
      # Parámetros para aumento de datos si es necesario
      if self.augment:
        self.datagen = ImageDataGenerator(
            rotation_range=90,
            width_shift_range=0.2,
            height_shift_range=0.2,
            zoom_range=0.2,
            horizontal_flip=True,
            vertical_flip=True,

            shear_range=0.2,
            fill_mode='nearest',
            brightness_range=[0.8, 1.2]
        )
      else:
        self.datagen = None
 

  # Selección y carga de imágenes para cada batch
  def __getitem__(self, index):
    # Selecciona índices del batch a partir de los índices mezclados
    batch_indexes = self.indexes[
        (index * self.batch_size) : ((index + 1) * self.batch_size)
    ]

    # Rutas de las imágenes del batch
    # [2] https://stackoverflow.com/questions/50997928/typeerror-only-integer-scalar-arrays-can-be-converted-to-a-scalar-index-with-1d
    batch_paths = self.image_paths[batch_indexes.astype(int)]

    # A partir de las rutas, extraemos las categorías ('rock', 'paper' o 'scissors')
    batch_labels = [self.labels[path] for path in batch_paths]

    # Cargamos las imágenes
    X, y = self.__data_generation(batch_paths, batch_labels)
    return X, y


  # Número de batches por época
  def __len__(self):
      return int(self.data_length // self.batch_size)


  # --- Atributos opcional: actualización cada tras época ---
  def on_epoch_end(self):
      # Actualizar índices tras cada época
      self.indexes = np.arange(self.data_length)
      if self.shuffle:
          np.random.shuffle(self.indexes)


  # --- Otros atributos ---
  def __data_generation(self, batch_paths, batch_labels):
      # Inicializamos salidas
      X = np.empty((self.batch_size, *self.image_size, 3))
      y = np.empty((self.batch_size), dtype=int) # debe ser int (hay que mapear clases)

      for i, (path, label) in enumerate(zip(batch_paths, batch_labels)):
          # Cargamos la imagen en path
          img = cv2.imread(path)
          img = cv2.resize(img, self.image_size) # Preprocesado: re-escalado de la imagen

          # Aumento de datos
          # [3] https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
          if self.augment:
              # Generar transformación aleatoria con los parámetros fijados en __init__
              img = self.datagen.random_transform(x=img)

          X[i,] = img / 255.0
          y[i] = self.class_mapping[label] # Mapeo de la clase de la imagen (continuación de corrección de Colab AI)

      return X, y


### Carga de datos <a class="anchor" id="carga-de-datos"></a>

In [ ]:
# --- Preparamos los datos ---
input_path = 'archive' # ruta en local
labels = {}
image_paths = []

# Create a dictionary mapping image paths to their labels
for clase in ['rock', 'paper', 'scissors']:
    directorio_clase = os.path.join(input_path, clase) # dirección de carpeta con fotos

    # Recorremos lista de imagenes en la carpeta
    for fname in os.listdir(directorio_clase):
      # Diccionario con key = ruta y value = clase
        ruta = os.path.join(directorio_clase, fname)
        image_paths.append(ruta)
        labels[ruta] = clase

# Separamos en train / test
train, test_val = train_test_split(image_paths, test_size=0.2, random_state=33)
validation, test = train_test_split(test_val, test_size=0.7, random_state=33)

In [ ]:
# --- Cargamos la clase Sequence para cada conjunto de datos ---
# Parámetros para cargar las imágenes (clase Sequence)
batch_size = 8 # Tamaño de cada batch (8 imágenes simultáneamente)
image_size = (128,128)  # Tamaño de las imágenes que le damos al modelo

# Objetos para secuenciar las imágenes de cada partición de los datos
train_seq = RockPaperScissorsSequence(image_paths=train,
                                      labels=labels,
                                      batch_size=batch_size,
                                      image_size=image_size,
                                      augment=True, # aumentamos los datos para dar generalidad al modelo
                                      shuffle=True)

validation_seq = RockPaperScissorsSequence(image_paths=validation,
                                           labels=labels,
                                           batch_size=batch_size,
                                           image_size=image_size,
                                           augment=False, # no modificamos las imágenes de validación
                                           shuffle=True) # hay que mezclar, se reutilizan en cada época

test_seq = RockPaperScissorsSequence(image_paths=test,
                                     labels=labels,
                                     batch_size=1,
                                     image_size=image_size,
                                     augment=False, # no modificamos las imágenes de test
                                     shuffle=False) # no es necesario mezclar, porque no se reutilizan

## Construcción y entrenamiento de las redes neuronales propuestas <a class="anchor" id="red-neuronal"></a>

Tenemos varias técnicas para construir redes neuronales que predigan con precisión los datos. La primera que vamos a ver en este trabajo es el "Transfer Learning", que consiste en cargar los pesos de una red neuronal, entrenada con otro conjunto de datos. A esta, le quitaremos las capas superiores (las cercanas a la salida) para, a continuación, añadir las que requiere nuestro conjunto de datos, que al menos será una capa densa de 3 neuronas.

Por otro lado, hemos construido un modelo siguiendo las pautas de [5], que podemos resumir en que debemos empezar por el modelo más simple posible y, a partir de él, construir uno más complejo si nuestro sistema lo necesita.

En cada caso, veremos dos aproximaciones al problema.

### Transfer Learning <a class="anchor" id="transfer-learning"></a>

#### VGG16 <a class="anchor" id="vgg16"></a>

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(
    tf.keras.applications.VGG16(weights=None, include_top=False, input_shape=(*image_size, 3)),
    to_file="images/VGG16.jpeg"
          )

In [ ]:
modelVGG16 = m.modeloVGG16(image_size=image_size)

modelVGG16.summary()
f.memoria_modelo(np.sum(
    [np.prod(v.get_shape().as_list()) for v in modelVGG16.trainable_variables]
), tipo='entrenables')

In [ ]:
# --- Preparamos el entrenamiento del modelo ---
# [6] https://stackoverflow.com/questions/58565394/what-is-the-difference-between-sparse-categorical-crossentropy-and-categorical-c
modelVGG16.compile(optimizer=tf.keras.optimizers.Adam(1e-3), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Detener el entrenamiento si no hay mejoras significativas (loss)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# --- Entrenamiento ----
# Train the model
modelVGG16.fit(train_seq, validation_data=validation_seq, # datos de entrenamiento y validación
          epochs=10,
          callbacks = [early_stopping, PlotLossesKerasTF()])

In [ ]:

# --- Fine Tuning ---
# Liberamos algunas capas para hacer fine tuning
capas_vgg_descongeladas = 5
for layer in modelVGG16.get_layer('vgg16').layers[-capas_vgg_descongeladas:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

# Preparamos el entrenamiento del modelo
# [6] https://stackoverflow.com/questions/58565394/what-is-the-difference-between-sparse-categorical-crossentropy-and-categorical-c
modelVGG16.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model
modelVGG16.fit(train_seq, validation_data=validation_seq, # datos de entrenamiento y validación
          epochs=10,
          callbacks = [early_stopping, PlotLossesKerasTF()])

#### ResNet <a class="anchor" id="resnet"></a>

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(
    tf.keras.applications.ResNet101V2(weights=None, include_top=False, input_shape=(*image_size, 3)),
    dpi=50,
    to_file="images/ResNet.jpeg"
          )

In [ ]:
modelResNet = m.modeloResNet(image_size=image_size)

modelResNet.summary()
f.memoria_modelo(np.sum(
    [np.prod(v.get_shape().as_list()) for v in modelResNet.trainable_variables]
), tipo='entrenables')

In [ ]:
# --- Preparamos el entrenamiento del modelo --- 
# [6] https://stackoverflow.com/questions/58565394/what-is-the-difference-between-sparse-categorical-crossentropy-and-categorical-c
modelResNet.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Parar el entrenamiento si no hay mejoras significativas (loss)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# --- Entrenamiento ----
# Train the model
modelResNet.fit(train_seq, validation_data=validation_seq, # datos de entrenamiento y validación
          epochs=10,
          callbacks = [early_stopping, PlotLossesKerasTF()])

In [ ]:
# --- Fine Tuning ---
# Liberamos algunas capas para hacer fine tuning
capas_resnet_descongeladas = 5
for layer in modelResNet.get_layer('resnet101v2').layers[-capas_resnet_descongeladas:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

# Preparamos el entrenamiento del modelo
# [6] https://stackoverflow.com/questions/58565394/what-is-the-difference-between-sparse-categorical-crossentropy-and-categorical-c
modelResNet.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model
modelResNet.fit(train_seq, validation_data=validation_seq, # datos de entrenamiento y validación
          epochs=10,
          callbacks = [early_stopping, PlotLossesKerasTF()])

### Modelos propios  <a class="anchor" id="modelos-propios"></a>

#### Modelo sin regularización Ridge <a class="anchor" id="sin-regularizacion"></a>

In [ ]:
model = m.modelo_sinRegularizacion(image_size=image_size)

model.summary()
f.memoria_modelo(np.sum(
    [np.prod(v.get_shape().as_list()) for v in model.trainable_variables]
), tipo='entrenables')

In [ ]:
plot_model(model)

In [ ]:
# --- Preparamos el entrenamiento del modelo ---
# [6] https://stackoverflow.com/questions/58565394/what-is-the-difference-between-sparse-categorical-crossentropy-and-categorical-c
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3),
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Parar el entrenamiento si no hay mejoras significativas (loss)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# --- Entrenamiento ----
# Train the model
model.fit(train_seq, validation_data=validation_seq, # datos de entrenamiento y validación
          epochs=30,
          callbacks = [early_stopping, PlotLossesKerasTF()])

In [ ]:
# --- Fine Tuning ---
# Preparamos el entrenamiento del modelo
# [6] https://stackoverflow.com/questions/58565394/what-is-the-difference-between-sparse-categorical-crossentropy-and-categorical-c
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model
model.fit(train_seq, validation_data=validation_seq, # datos de entrenamiento y validación
          epochs=30,
          callbacks = [early_stopping, PlotLossesKerasTF()])

Con una tasa de aprendizaje mayor:

In [ ]:
model = m.modelo_sinRegularizacion(image_size=image_size)

# --- Preparamos el entrenamiento del modelo ---
# [6] https://stackoverflow.com/questions/58565394/what-is-the-difference-between-sparse-categorical-crossentropy-and-categorical-c
model.compile(optimizer=tf.keras.optimizers.Adam(5e-2),
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Parar el entrenamiento si no hay mejoras significativas (loss)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# --- Entrenamiento ----
# Train the model
model.fit(train_seq, validation_data=validation_seq, # datos de entrenamiento y validación
          epochs=30,
          callbacks = [early_stopping, PlotLossesKerasTF()])

#### Modelo con regularización Ridge <a class="anchor" id="con-regularizacion"></a>

In [ ]:
modelReg = m.modelo_conRegularizacion(image_size=image_size)

modelReg.summary()
f.memoria_modelo(np.sum(
    [np.prod(v.get_shape().as_list()) for v in modelReg.trainable_variables]
), tipo='entrenables')

In [ ]:
plot_model(modelReg, to_file="images/mi_modelo.jpeg")

In [ ]:
# --- Preparamos el entrenamiento del modelo ---
# [6] https://stackoverflow.com/questions/58565394/what-is-the-difference-between-sparse-categorical-crossentropy-and-categorical-c
modelReg.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Parar el entrenamiento si no hay mejoras significativas (loss)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# --- Entrenamiento ----
# Train the model
modelReg.fit(train_seq, validation_data=validation_seq, # datos de entrenamiento y validación
          epochs=30,
          callbacks = [early_stopping, PlotLossesKerasTF()])

In [ ]:
# --- Fine Tuning ---
# Preparamos el entrenamiento del modelo
# [6] https://stackoverflow.com/questions/58565394/what-is-the-difference-between-sparse-categorical-crossentropy-and-categorical-c
modelReg.compile(optimizer=tf.keras.optimizers.Adam(1e-5),
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# Train the model
modelReg.fit(train_seq, validation_data=validation_seq, # datos de entrenamiento y validación
          epochs=30,
          callbacks = [early_stopping, PlotLossesKerasTF()])

# Resultados <a class="anchor" id="resultados"></a>

## Modelo propio con regularización

### Precisión, recall, F1-score y confusion matrix

In [ ]:
# [13] https://www.kaggle.com/code/fathyalin/rock-paper-scissors-tensorflow-with-real-photos/notebook

predictions = np.argmax(modelReg.predict(test_seq), axis=1)
reales = [test_seq.class_mapping[test_seq.labels[path]] for path in test_seq.image_paths]

acc = modelReg.evaluate(test_seq, verbose=0)[1]
otras_medidas = pd.DataFrame(skm.precision_recall_fscore_support(reales, predictions), 
                             columns=("Piedra", "Papel", "Tijeras"), 
                             index=["Precision", "Recall", "F1-score", "Número de casos"]).transpose()

In [ ]:
cm = confusion_matrix(y_true=reales, y_pred=predictions, labels=[0,1,2])

plt.figure(figsize=(8,8))
sns.heatmap(cm, annot=True, fmt='g', vmin=0, cbar=False)
plt.xticks(ticks=[0.5, 1.5, 2.5], labels=['Piedra','Papel','Tijeras'])
plt.yticks(ticks=[0.5, 1.5, 2.5], labels=['Piedra','Papel','Tijeras'])
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de confusión")
plt.savefig('images/propia/confusion_matrix.jpeg', bbox_inches='tight')
plt.show()

print('Medidas de rendimiento\n', 
      f'Accuracy (sobre test): {np.round(acc*100, 1)}%\n',
     otras_medidas)

### Clasificación de imágenes en correctas e incorrectas

La clasificación en una de las categorías depende de si esta correcta o incorrectamente clasificada.

In [ ]:
# Pre-cargamos las lista de índices de imágenes correcta e incorrectamente clasificadas
lista_idx_correctos = []
lista_idx_incorrectos = []

# Cargamos batches de 1 imagen para poder usarlas todas
# Si tenemos 307 imágenes y tomamos batches de 100 se pierden las 7 últimas
test_seq.batch_size = 1
n_imgs = len(test_seq)

for idx in range(n_imgs):
    # Mostrar el progreso
    print('\r', f'{idx+1} / {n_imgs}', end='')
    
    # Cargamos la imágen, su clase real y la predicha
    im, clase_real = test_seq[idx]
    im = np.copy(im)
    clase_pred = np.argmax(modelReg.predict(im, verbose=0), axis=1)[0]

    # Comprobamos si la imágen se clasifica correctamente o no
    if clase_real == clase_pred:
        lista_idx_correctos.append(idx)
    else:
        lista_idx_incorrectos.append(idx)

### Casos incorrectamente clasificados

#### Análisis de predicciones incorrectas
Práctica 6 (Grad-CAM)

In [ ]:
from importlib import reload
reload(f)

In [ ]:
# Índices de imágenes incorrectas desordenado
np.random.shuffle(lista_idx_incorrectos)

# Plot
images = f.imagenes_test(modelReg, test_seq, lista_idx_incorrectos)
f.show_images(images)

#### Heatmaps de casos incorrectamente clasificados

In [ ]:
if lista_idx_incorrectos:
    f.show_heatmaps(images, modelReg, clasificacion='propia/clasificacion_incorrecta')
else:
    print("No hay imágenes incorrectamente clasificadas!!")

### Casos correctamente clasificados

#### Análisis de predicciones incorrectas
Práctica 6 (Grad-CAM)

In [ ]:
# Índices de imágenes incorrectas desordenado
np.random.shuffle(lista_idx_correctos)

# Plot
images = f.imagenes_test(modelReg, test_seq, lista_idx_correctos)
f.show_images(images)

#### Heatmaps de casos correctamente clasificados

In [ ]:
f.show_heatmaps(images, modelReg, clasificacion='propia/clasificacion_correcta')

### Visualización de los filtros

#### Visualización de las activaciones de los filtros convolucionales

In [ ]:
from importlib import reload
reload(f)

f.visualizacion_filtros(modelReg, n_rows_grid=8, clasificacion='propia')

#### Kernels y mapas de características de la primera capa convolucional sobre una imagen correctamente clasificada

https://www.kaggle.com/code/sanjitschouhan/visualizing-conv2d-output

In [ ]:
np.random.seed(33)

# Seleccionamos una imagen (de las bien clasificadas) aleatoriamente
np.random.shuffle(lista_idx_correctos)
idx = lista_idx_correctos[0]
im, clase_real = test_seq[idx]
im = np.copy(im)

# Plot
# [14] https://stackoverflow.com/questions/50630825/matplotlib-imshow-distorting-colors
plt.imshow(im[0][...,::-1])
plt.title(f"Clase real: {('Piedra', 'Papel', 'Tijeras')[clase_real[0]]}")
plt.savefig('images/propia/ejemplo_imagen_correcta.jpeg', 
            bbox_inches='tight')
plt.show()

In [ ]:
# Obtener los kernels de la primera capa convolucional
primera_conv2d = modelReg.get_layer('conv2d_entrada')
kernels, _ = primera_conv2d.get_weights()
n_kernels = kernels.shape[-1]

plt.figure(figsize=(20,100))

# Número de filas a dibujar
n_filters = kernels.shape[-1]
n_rows = n_filters // 3 + 1

for idx in range(n_kernels):
  # Obtener el kernel
  kernel = np.array(kernels[:,:,:,idx])
  kernel_in = tf.constant(
      np.reshape(kernel, (*kernel.shape, 1))
  )

  # Imagen
  image_in = tf.constant(
      np.reshape(im, (1,128,128,3)), dtype=tf.float32
      )

  # Aplicamos el kernel a la imagen correctamente clasificada
  salida = tf.nn.conv2d(image_in,
                        filters = kernel_in,
                        strides = [1, 1, 1, 1],
                        padding = 'SAME')
    
  # Plot
  # [15] https://stackoverflow.com/questions/49643907/clipping-input-data-to-the-valid-range-for-imshow-with-rgb-data-0-1-for-floa
  plt.subplot(n_rows,6, 2*idx+1)
  plt.imshow((kernel * 255).astype(np.uint8))
  plt.title("Filter Kernel "+str(idx+1))
  plt.xticks([])
  plt.yticks([])

  plt.subplot(n_rows,6, 2*idx+2)
  plt.imshow(np.reshape(salida * 255, (128,128)).astype(np.uint8), cmap='gray')

  plt.title("Kernel Output "+str(idx+1))
  plt.xticks([])
  plt.yticks([])

plt.savefig('images/propia/activaciones/kernels_primera_convolucional.jpeg', bbox_inches='tight')
plt.show()


## VGG16

### Precisión, recall, F1-score y confusion matrix

In [ ]:
# [13] https://www.kaggle.com/code/fathyalin/rock-paper-scissors-tensorflow-with-real-photos/notebook

predictions = np.argmax(modelVGG16.predict(test_seq), axis=1)
reales = [test_seq.class_mapping[test_seq.labels[path]] for path in test_seq.image_paths]

acc = modelVGG16.evaluate(test_seq, verbose=0)[1]
otras_medidas = pd.DataFrame(skm.precision_recall_fscore_support(reales, predictions), 
                             columns=("Piedra", "Papel", "Tijeras"), 
                             index=["Precision", "Recall", "F1-score", "Número de casos"]).transpose()

In [ ]:
cm = confusion_matrix(y_true=reales, y_pred=predictions, labels=[0,1,2])

plt.figure(figsize=(8,8))
sns.heatmap(cm, annot=True, fmt='g', vmin=0, cbar=False)
plt.xticks(ticks=[0.5, 1.5, 2.5], labels=['Piedra','Papel','Tijeras'])
plt.yticks(ticks=[0.5, 1.5, 2.5], labels=['Piedra','Papel','Tijeras'])
plt.xlabel("Predicción")
plt.ylabel("Real")
plt.title("Matriz de confusión")
plt.savefig('images/vgg16/confusion_matrix.jpeg', bbox_inches='tight')
plt.show()

print('Medidas de rendimiento\n', 
      f'Accuracy (sobre test): {np.round(acc*100, 2)}%\n',
     otras_medidas)

### Clasificación de imágenes en correctas e incorrectas

La clasificación en una de las categorías depende de si esta correcta o incorrectamente clasificada.

In [ ]:
# Pre-cargamos las lista de índices de imágenes correcta e incorrectamente clasificadas
lista_idx_correctos = []
lista_idx_incorrectos = []

# Cargamos batches de 1 imagen para poder usarlas todas
# Si tenemos 307 imágenes y tomamos batches de 100 se pierden las 7 últimas
test_seq.batch_size = 1
n_imgs = len(test_seq)

for idx in range(n_imgs):
    # Mostrar el progreso
    print('\r', f'{idx+1} / {n_imgs}', end='')
    
    # Cargamos la imágen, su clase real y la predicha
    im, clase_real = test_seq[idx]
    im = np.copy(im)
    clase_pred = np.argmax(modelVGG16.predict(im, verbose=0), axis=1)[0]

    # Comprobamos si la imágen se clasifica correctamente o no
    if clase_real == clase_pred:
        lista_idx_correctos.append(idx)
    else:
        lista_idx_incorrectos.append(idx)

### Casos incorrectamente clasificados

#### Análisis de predicciones incorrectas
Práctica 6 (Grad-CAM)

In [ ]:
from importlib import reload
reload(f)

In [ ]:
# Índices de imágenes incorrectas desordenado
np.random.shuffle(lista_idx_incorrectos)

# Plot
images = f.imagenes_test(modelVGG16, test_seq, lista_idx_incorrectos)
f.show_images(images)

#### Heatmaps de casos incorrectamente clasificados

In [ ]:
from importlib import reload
reload(f)

In [ ]:
if lista_idx_incorrectos:
    f.show_heatmaps(images, modelVGG16.get_layer('vgg16'), n_finales=capas_vgg_descongeladas, clasificacion='vgg16/clasificacion_incorrecta')
else:
    print("No hay imágenes incorrectamente clasificadas!!")

### Casos correctamente clasificados

#### Análisis de predicciones incorrectas
Práctica 6 (Grad-CAM)

In [ ]:
# Índices de imágenes incorrectas desordenado
np.random.shuffle(lista_idx_correctos)

# Plot
images = f.imagenes_test(modelVGG16, test_seq, lista_idx_correctos)
f.show_images(images)

#### Heatmaps de casos correctamente clasificados

In [ ]:
f.show_heatmaps(images, modelVGG16.get_layer('vgg16'), n_finales=0, clasificacion='vgg16/clasificacion_correcta')

### Visualización de los filtros

#### Visualización de las activaciones de los filtros convolucionales

In [ ]:
from importlib import reload
reload(f)

f.visualizacion_filtros(modelVGG16.get_layer('vgg16'), n_rows_grid=8, n_top_layers=capas_vgg_descongeladas, clasificacion='vgg16')

#### Kernels y mapas de características de la primera capa convolucional sobre una imagen correctamente clasificada

https://www.kaggle.com/code/sanjitschouhan/visualizing-conv2d-output

In [ ]:
np.random.seed(33)

# Seleccionamos una imagen (de las bien clasificadas) aleatoriamente
np.random.shuffle(lista_idx_correctos)
idx = lista_idx_correctos[0]
im, clase_real = test_seq[idx]
im = np.copy(im)

# Plot
# [14] https://stackoverflow.com/questions/50630825/matplotlib-imshow-distorting-colors
plt.imshow(im[0][...,::-1])
plt.title(f"Clase real: {('Piedra', 'Papel', 'Tijeras')[clase_real[0]]}")
plt.show()

In [ ]:
# Obtener los kernels de la primera capa convolucional
primera_conv2d = modelVGG16.get_layer('vgg16').get_layer('block1_conv1')
kernels, _ = primera_conv2d.get_weights()
n_kernels = kernels.shape[-1]

plt.figure(figsize=(20,100))

# Número de filas a dibujar
n_filters = kernels.shape[-1]
n_rows = n_filters // 3 + 1

for idx in range(n_kernels):
  # Obtener el kernel
  kernel = np.array(kernels[:,:,:,idx])
  kernel_in = tf.constant(
      np.reshape(kernel, (*kernel.shape, 1))
  )

  # Imagen
  image_in = tf.constant(
      np.reshape(im, (1,128,128,3)), dtype=tf.float32
      )

  # Aplicamos el kernel a la imagen correctamente clasificada
  salida = tf.nn.conv2d(image_in,
                        filters = kernel_in,
                        strides = [1, 1, 1, 1],
                        padding = 'SAME')
    
  # Plot
  # [15] https://stackoverflow.com/questions/49643907/clipping-input-data-to-the-valid-range-for-imshow-with-rgb-data-0-1-for-floa
  plt.subplot(n_rows,6, 2*idx+1)
  plt.imshow((kernel * 255).astype(np.uint8))
  plt.title("Filter Kernel "+str(idx+1))
  plt.xticks([])
  plt.yticks([])

  plt.subplot(n_rows,6, 2*idx+2)
  plt.imshow(np.reshape(salida * 255, (128,128)).astype(np.uint8), cmap='gray')

  plt.title("Kernel Output "+str(idx+1))
  plt.xticks([])
  plt.yticks([])

plt.savefig('images/vgg16/activaciones/kernels_primera_convolucional.jpeg', 
            bbox_inches='tight')
plt.show()
